In [8]:
from customerManager import CustomerManager
customer_path = "./config/customer_classes.json"
cManager = CustomerManager(file_path = customer_path)

In [1]:
import numpy as np
from scipy.special import erfc

class Learner():
    def __init__(self):
        self.n_bids = 10
        self.n_price= 13
        self.n_arms =  self.n_bids*self.n_price
        self.bid = np.linspace(1.0, 10.0, self.n_bids)
        self.bid = np.linspace(3.0, 15.0, self.n_price)
        self.t = 0 
        
        
        self.rewards_per_arm = x = [[[] for i in range(self.n_price)] for i in range(self.n_bids)]
        self.collected_rewards = []
    
    def update_observations(self, pulled_arm_bid,pulled_arm_price, reward):
        self.rewards_per_arm[pulled_arm_bid][pulled_arm_price].append(reward)
        self.collected_rewards.append(reward)

class TS_Learner(Learner):
    def __init__(self, safety_t = 0.2):
        super().__init__()
        self.safety_t = safety_t
        #Mean and Std
        self.normal_parameters = np.zeros((self.n_bids,self.n_price, 2))

    def pull_arm(self):
        if self.t < (2*self.n_arms): return (self.t % self.n_bids) + 1,(self.t % self.n_price) +3 # controllar ebene che sia corretto

        #Safety constraint
        #Alternative implementation safety constraints using MATH         
        arg_max = [[] for i in range(self.n_bids)]
        for b_index in range(self.n_bids) : 
            for p in self.normal_parameters[b_index]:
                if 0.5 * erfc(p[0]/(np.sqrt(2)*p[1])) > self.safety_t :
                    arg_max[b_index].append(0.0) 
                else : 
                    arg_max[b_index].append(np.random.normal(p[0], p[1]))
        
        result = np.where(arg_max == np.amax(arg_max))
        listOfCordinates = list(zip(result[0], result[1]))
      
        return listOfCordinates[0][0] + 1,listOfCordinates[0][1] + 3  #index Bid and price conversion

    def update(self, pulled_arm_bid,pulled_arm_price, reward):
        self.t += 1
        pulled_arm_bid -= 1  #Bid index conversion
        pulled_arm_price -= 3  #price index conversion
        times_pulled = max(1, len(self.rewards_per_arm[pulled_arm_bid][pulled_arm_price]))
        self.update_observations(pulled_arm_bid,pulled_arm_price, reward)
        #Empirical mean
        self.normal_parameters[pulled_arm_bid][pulled_arm_price][0] = (self.normal_parameters[pulled_arm_bid][pulled_arm_price][0] * (times_pulled - 1) + reward ) / times_pulled
        #Empirical std
        #NOt gooooood Should depend on the number of sampl (inversionally proportional)
        self.normal_parameters[pulled_arm_bid][pulled_arm_price][1] = np.sqrt(sum((self.normal_parameters[pulled_arm_bid][pulled_arm_price][0] - self.rewards_per_arm[pulled_arm_bid][pulled_arm_price])**2)/times_pulled**2)


In [4]:
opt_bids = [3.8622484787564275 , 2.1216094606111944,  2.347134281066495]
opt_price =6.321089806558111

In [10]:
from environment import EnvironmentSingleClass
from tqdm import tqdm

T = 365
env = EnvironmentSingleClass(noise_variance= 0.05)

ts_learners= [TS_Learner(),TS_Learner(),TS_Learner()]

regrets =[[],[],[]]

sols = [[],[],[]]

for t in tqdm(range(T)):
    #TS learner 
    for c in range(3):
        bid,price = ts_learners[c].pull_arm()
        bids = [bid,bid,bid]
        sols[c].append([bid,price])
        reward = env.round(bids,price,c_id = c)
        ts_learners[c].update(bid,price, reward)
        regrets[c].append(env.round(opt_bids, opt_price,c, noise = False) - reward)



100%|██████████| 365/365 [00:00<00:00, 1077.55it/s]


In [12]:
import matplotlib.pyplot as plt 

plt.figure(0)
plt.xlabel("t")
plt.ylabel("Regret")
plt.plot(regrets[0] , 'r')
plt.plot(regrets[1] , 'g')
plt.plot(regrets[2] , 'b')
plt.legend(["Class_0","Class_1","Class_2" ])
plt.show()

In [18]:
regret[-10:]

[37.59922617119673,
 34.63439454815489,
 316.8922696646641,
 -224.98479741958727,
 795.7104552765159,
 -544.2085350891748,
 910.7976620629947,
 467.53796846441946,
 1001.5322992813499,
 140.82842129683922]

In [19]:
sols[-10:]

[[3, 7],
 [3, 7],
 [2, 7],
 [2, 6],
 [3, 7],
 [3, 7],
 [5, 6],
 [3, 7],
 [3, 7],
 [3, 7]]